In [ ]:
import os, sys, pdb
import numpy as np
import random
import torch

import math

from tqdm import tqdm as progress_bar

from utils import set_seed, setup_gpus, check_directories
from dataloader import get_dataloader, check_cache, prepare_features, process_data, prepare_inputs
from load import load_data, load_tokenizer
from arguments import params
from model import IntentModel, SupConModel, CustomModel
from torch import nn
import torch.optim as optim



In [ ]:
train_dataloader = get_dataloader(args=args, dataset=datasets)

In [16]:
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased') #,    return_dict = True
text = "The capital of France, " + tokenizer.mask_token + ", contains the Eiffel Tower."
input = tokenizer.encode_plus(text, return_tensors = "pt")
mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
output = model(**input)

# print(f"output: {output}")
last_hidden_states = output.last_hidden_state
print(last_hidden_states)

logits = output.logits
softmax = F.softmax(logits, dim = -1)
mask_word = softmax[0, mask_index, :]
top_10 = torch.topk(mask_word, 10, dim = 1)[1][0]
for token in top_10:
   word = tokenizer.decode([token])
   new_sentence = text.replace(tokenizer.mask_token, word)
   print(new_sentence)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


AttributeError: 'MaskedLMOutput' object has no attribute 'last_hidden_state'

In [26]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

# last_hidden_states = outputs.last_hidden_state
last_hidden_states = outputs[0]
print(last_hidden_states.shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 8, 768])


In [5]:
test_Example = torch.randn(3,5,10)

In [7]:
test_Example[:,0,:]

tensor([[-0.3385,  0.9223,  1.4355,  0.9748,  0.8270, -0.7024, -0.7909,  0.7364,
         -1.2839, -0.6640],
        [-1.0999, -0.8466,  0.1845, -1.9995,  0.8334,  0.6542, -0.2485, -0.7534,
          1.6890, -0.9724],
        [ 0.8050,  0.1781, -0.0378, -0.7967, -1.1400,  1.2242,  0.0721, -2.3814,
          0.6954, -1.5368]])

In [8]:
test_Example[0][0]

tensor([-0.3385,  0.9223,  1.4355,  0.9748,  0.8270, -0.7024, -0.7909,  0.7364,
        -1.2839, -0.6640])